In [ ]:
from methods.slab_topo import inside_topo,surface_topo
from ase.io import read
filename = ''
atoms = read(filename)
inside_sites = inside_topo(atoms,bond_len=1.65)
print("inside_sites:",len(inside_sites))
surface_sites = surface_topo(atoms,bond_len=1.65,both_surface=True)
print("surface_sites:",len(surface_sites))


In [26]:
import random
import numpy as np 
def calculate_distance(cell,point1, point2):
    """The function to calculated distance 
        for dealing with periodicity

    Parameters:
    -----------
    cell : ndarray
        The cell in input atoms.
    points1 , points2 : ndarray
        The positions to calculate distance with periodicity 
        
    """
    a_vec = cell[0]
    b_vec = cell[1]
    dis = 10.
    neighbors = []
    for da in range(-1,2):
        for db in range(-1,2):
            neighbor_position = point1 + da * a_vec + db * b_vec
            neighbors.append(neighbor_position)
    for n in neighbors:
        if dis > np.linalg.norm(n-point2):
            dis = np.linalg.norm(n-point2)
    return dis

def select_points(points, num_points, min_distance,cell):
    """
    The functino is Used to carry out the search for picking sites.

    Parameters:
    -----------
    points : list of ndarray
        All possible potential sites should be entered
    
    num_points: int 
        Number of adsorbents

    min_distance : float 
        The minimum distance between two neighboring adsorbents

    cell : ndarray
        The cell in input atoms.
    """
    selected_points = []
    
    while len(selected_points) < num_points:
        point = random.choice(points)
        if all(calculate_distance(cell,point, selected_point) >= min_distance for selected_point in selected_points):
            selected_points.append(point)
    return selected_points

In [ ]:
from ase.neighborlist import natural_cutoffs, NeighborList
import networkx as nx
from networkx.algorithms import isomorphism

def is_unique(graph,unique_graphs):
    """Determine if the current input graph is unique

    Parameters:
    -----------
    graph : networkx.Graph object
        The graph for determining uniqueness

    unique_graphs: list of networkx.Graph object
        List of saved graphs with unique 
    """
    if unique_graphs==[]:
         return True
    for i,unique_graph in enumerate(unique_graphs):
        GM = isomorphism.GraphMatcher(graph, unique_graph, node_match=isomorphism.categorical_node_match('symbol', ''))
        if GM.is_isomorphic():
            return False
    return True

def get_graph(atoms):
    #Get Periodicity Graph
    cutoffs = natural_cutoffs(atoms)
    positions = atoms.get_positions()
    cell = atoms.get_cell()
    G = nx.Graph()
    symbols = atoms.symbols                               
    G.add_nodes_from([(i, {'symbol': symbols[i]}) 
                          for i in range(len(symbols))])
    for i in range(len(positions)):
        for j in range(i + 1, len(positions)):
            distance = calculate_distance(cell,positions[i],positions[j])
            if distance < cutoffs[i]+cutoffs[j]:
                G.add_edge(i, j)
    return G

def wl(graph):
        #The function is to turn input graph into wl-graph by wl conversion
        node_symbols = nx.get_node_attributes(graph, 'symbol')
        num_iterations = 3
        for _ in range(num_iterations):
            new_symbols = {}
            for node in graph.nodes():
                symbol = node_symbols[node]
                neighbor_symbols = [node_symbols[neighbor] for neighbor in graph.neighbors(node)]
                combined_symbol = symbol + ''.join(sorted(neighbor_symbols))
                new_symbols[node] = combined_symbol
            node_symbols = new_symbols

        new_graph = nx.Graph()
        for node, symbol in node_symbols.items():
            new_graph.add_node(node, symbol=symbol)

        return new_graph

In [ ]:
from ase import Atoms
sites = inside_sites + surface_sites
num_ad = 3
num_configuration = 10
symbol = 'H'
configurations = []
unique_graphs = []
for i in range(num_configuration):
    structure = atoms.copy()
    points = select_points(sites,num_ad,1.5)
    for p in points:
        ad = Atoms(symbols=symbol,positions=[p])
        structure += ad
    graph = get_graph(structure)
    if is_unique(graph,unique_graphs) == True:
        unique_graphs.append(graph)
        configurations.append(structure)
print(configurations)